In [1]:
from sympy import *
from sympy import init_printing
init_printing()
uM, uL, uR, sL, sR, ssL, ssR, xi, nu, theta = symbols('u_M u_L u_R s_L s_R ss_L ss_R xi nu theta')
uML, uLL, sLL, ssLL = symbols('u_M^L u_L^L s_L^L ss_L^L')
c0, c1, c2, c3, c4, c5, c6 = symbols('c_0 c_1 c_2 c_3 c_4 c_5 c_6')
u = c0 + c1*xi + c2*xi**2 + c3*xi**3 + c4*xi**4 + c5*xi**5 + c6*xi**6
DuDxi = diff(u,xi)
D2uDxi2 = diff(u,xi,2)
Iu = Integral(u,(xi,0,1))
Iu = simplify(Iu)

# setup and solve the equation
eqns = [u.subs(xi,0) - uL,
        u.subs(xi,1) - uR,
        DuDxi.subs(xi,0) - sL,
        DuDxi.subs(xi,1) - sR,
        D2uDxi2.subs(xi,0) - ssL,
        D2uDxi2.subs(xi,1) - ssR,
                    1*Iu - uM]
vars = [c0, c1, c2, c3, c4, c5, c6]
sol = solve(eqns, vars)
u = u.subs([(c0,sol[c0]),
            (c1,sol[c1]),
            (c2,sol[c2]),
            (c3,sol[c3]),
            (c4,sol[c4]),
            (c5,sol[c5]),
            (c6,sol[c6])])
DuDxi = diff(u,xi)
D2uDxi2 = diff(u,xi,2)

u = separatevars(u)
u = collect(u, [uM, uL, uR, sL, sR, ssL, ssR])

phiuM, phiuL, phiuR, phisL, phisR, phissL, phissR = symbols('phi_uM phi_uL phi_uR phi_sL phi_sR phi_ssL phi_ssR')
phiuM = u.coeff(uM)
phiuL = u.coeff(uL)
phiuR = u.coeff(uR)
phisL = u.coeff(sL)
phisR = u.coeff(sR)
phissL = u.coeff(ssL)
phissR = u.coeff(ssR)

psi = symbols('psi')
a, b, c, d, e, f = symbols('a b c d e f')
psi = 1 + a*xi + b*xi**2 + c*xi**3 + d*xi**4 + e*xi**5 + f*xi**6

IuM  = integrate(expand(psi*phiuM, xi),(xi,0,1))
IuL  = integrate(expand(psi*phiuL, xi),(xi,0,1))
IuR  = integrate(expand(psi*phiuR, xi),(xi,0,1))
IsL  = integrate(expand(psi*phisL, xi),(xi,0,1))
IsR  = integrate(expand(psi*phisR, xi),(xi,0,1))
IssL = integrate(expand(psi*phissL, xi),(xi,0,1))
IssR = integrate(expand(psi*phissR, xi),(xi,0,1))

# setup and solve the equations set 1 (ssR)
eqns = [IuL, IsL, IssL, IuM, IuR, IsR]
vars = [a, b, c, d, e, f]
sol = solve(eqns, vars)
psissR = psi.subs([(a,sol[a]),
                   (b,sol[b]),
                   (c,sol[c]),
                   (d,sol[d]),
                   (e,sol[e]),
                   (f,sol[f])])

# setup and solve the equations set 2 (sR)
eqns = [IuL, IsL, IssL, IuM, IuR, IssR]
vars = [a, b, c, d, e, f]
sol = solve(eqns, vars)
psisR = psi.subs([(a,sol[a]),
                  (b,sol[b]),
                  (c,sol[c]),
                  (d,sol[d]),
                  (e,sol[e]),
                  (f,sol[f])])

# setup and solve the equations set 3 (uR)
eqns = [IuL, IsL, IssL, IuM, IsR, IssR]
vars = [a, b, c, d, e, f]
sol = solve(eqns, vars)
psiuR = psi.subs([(a,sol[a]),
                  (b,sol[b]),
                  (c,sol[c]),
                  (d,sol[d]),
                  (e,sol[e]),
                  (f,sol[f])])

# setup and solve the equations set 4 (uM)
eqns = [IuL, IsL, IssL, IuR, IsR, IssR]
vars = [a, b, c, d, e, f]
sol = solve(eqns, vars)
psiuM = psi.subs([(a,sol[a]),
                  (b,sol[b]),
                  (c,sol[c]),
                  (d,sol[d]),
                  (e,sol[e]),
                  (f,sol[f])])

# output results, order is important for building mass matrix in C++
data = {}
data['phi'] = [str(phiuL),
               str(phisL),
               str(phissL),
               str(phiuM),
               str(phiuR),
               str(phisR),
               str(phissR)]
data['psi'] = [str(psiuM),
               str(psiuR),
               str(psisR),
               str(psissR)]
data['diffpsi'] = [str(diff(psiuM,xi)), 
                   str(diff(psiuR,xi)),
                   str(diff(psisR,xi)),
                   str(diff(psissR,xi))]
import json
with open('json/DG3.json', 'w') as outfile:
    json.dump(data, outfile, indent=4)